In [1]:
import os
import cv2
import torch
import numpy as np
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, models
import traceback

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Torch empty cache
torch.cuda.empty_cache()

Using device: cuda


In [2]:
def find_last_convolutional_layer(model):
    """
    Recursively find the last convolutional layer in the model
    """
    last_conv_layer = None
    
    def _find_conv_layer(module):
        nonlocal last_conv_layer
        for name, child in module.named_children():
            if isinstance(child, (torch.nn.Conv2d, torch.nn.modules.conv._ConvNd)):
                last_conv_layer = child
            _find_conv_layer(child)
    
    _find_conv_layer(model)
    return last_conv_layer

In [3]:
def load_model(model_path, num_classes=5):
    # Determine model architecture based on filename
    filename = os.path.basename(model_path)
    
    # Mapping of model architectures
    model_architectures = {
        'convnext_tiny': models.convnext_tiny(weights=None),
        'mobilenetv2': models.mobilenet_v2(weights=None),
        'regnety_8gf': models.regnet_y_8gf(weights=None),
        'resnet50': models.resnet50(weights=None),
        'efficientnet': models.efficientnet_b0(weights=None)
    }
    
    # Select model architecture
    for arch_name, model_class in model_architectures.items():
        if arch_name in filename.lower():
            model = model_class
            break
    else:
        raise ValueError(f"Could not determine model architecture for {filename}")
    
    # Modify classifier for custom number of classes
    if 'convnext_tiny' in filename.lower():
        model.classifier[2] = torch.nn.Linear(in_features=model.classifier[2].in_features, out_features=num_classes)
    
    elif 'mobilenetv2' in filename.lower():
        model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=num_classes)
    
    elif 'regnety' in filename.lower():
        model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
    
    elif 'resnet50' in filename.lower():
        model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

    elif 'efficientnet' in filename.lower():
        model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=num_classes)
    else:
        raise ValueError(f"Unsupported model architecture in {filename}")
    
    # Load state dict
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    
    return model


In [4]:
def get_target_layer(model, filename):
    # Detailed handling for different model architectures
    if 'convnext_tiny' in filename.lower():
        # For ConvNeXt, use the last stage
        return model.stages[-1]
    
    elif 'efficientnet' in filename.lower():
        # For EfficientNet, use the last block in the network
        return model.features[-1]
    
    elif 'mobilenetv2' in filename.lower():
        # For MobileNetV2, use the last convolutional layer
        return model.features[-1]
    
    elif 'regnety' in filename.lower():
        # For RegNet, use the last block in the network
        return model.trunk_output[-1]
    
    elif 'resnet50' in filename.lower():
        # For ResNet, use the last convolutional layer of the last block
        return model.layer4[-1]
    
    else:
        raise ValueError(f"Could not determine target layer for {filename}")

In [5]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model.eval().to(device)
        self.target_layer = target_layer.to(device)
        self.activation = None
        self.gradients = None

        self.target_layer.register_forward_hook(self.forward_hook)
        self.target_layer.register_full_backward_hook(self.backward_hook)


    def forward_hook(self, module, input, output):
        self.activation = output.detach()

    def backward_hook(self, module, grad_in, grad_out):
        if grad_out[0] is not None:
            self.gradients = grad_out[0].detach()

        else:
            print(f"Warning: backward_hook received None gradients for module {module}. Make sure the target layer is part of the computational graph for the loss.")
            self.gradients = None

    def generate_heatmap(self, input_tensor,heatmap_method):
        input_tensor = input_tensor.to(device)
        if not input_tensor.requires_grad:
            input_tensor.requires_grad_(True)

        if not hasattr(self, 'forward_handle') or self.forward_handle is None:
             self.forward_handle = self.target_layer.register_forward_hook(self.forward_hook)
        if not hasattr(self, 'backward_handle') or self.backward_handle is None:
             self.backward_handle = self.target_layer.register_full_backward_hook(self.backward_hook)
        
        self.activation = None
        self.gradients = None
        self.model.zero_grad()
        
        output = self.model(input_tensor)

        target_class = output.argmax(dim=1).item()

        if self.activation is None:
            raise RuntimeError("Activation hook did not run. Check target_layer.")

        target_score = output[0, target_class]
        target_score.backward(retain_graph=False, create_graph=False)
        
        if self.gradients is None:
             raise RuntimeError("Backward hook did not capture gradients. Check target_layer involvement and backward pass.")

        if heatmap_method == 'default':
            weights = self.gradients.mean(dim=(2, 3), keepdim=True) # Global Average Pooling of gradients
            cam_raw = (weights * self.activation).sum(dim=1, keepdim=True) # Weighted sum of activations
            cam_raw = torch.relu(cam_raw) # Apply ReLU
            # Squeeze, move to CPU, convert to numpy AFTER ensuring it's not empty
            if cam_raw.numel() > 0:
                 cam = cam_raw.squeeze().cpu().numpy()
            else:
                 print("Warning: Raw CAM tensor is empty.")
                 # Handle appropriately, e.g., return zeros or raise error
                 h, w = self.activation.shape[2:] # Get spatial dims from activation
                 cam = np.zeros((h, w), dtype=np.float32)
        
        elif heatmap_method == 'guided':
            weights = self.gradients.mean(dim=(2, 3), keepdim=True)
            cam_raw = (weights * self.activation).sum(dim=1, keepdim=True)
            cam_raw = torch.relu(cam_raw)
            if cam_raw.numel() > 0:
                cam = cam_raw.squeeze().cpu().numpy()
            else:
                print("Warning: Raw CAM tensor is empty.")
                h, w = self.activation.shape[2:]
                cam = np.zeros((h, w), dtype=np.float32)
        
        elif heatmap_method == 'gradcam++':
            gradients_pow2 = self.gradients.pow(2)
            gradients_pow3 = self.gradients.pow(3)
            sum_act = self.activation.sum(dim=(2, 3), keepdim=True) # Sum over spatial dimensions

            # Adding epsilon for numerical stability
            eps = 1e-8
            alpha_denom = 2.0 * gradients_pow2 + sum_act * gradients_pow3 + eps
            alpha_num = gradients_pow2

            # Element-wise division, handling potential division by zero via epsilon
            alpha = alpha_num / alpha_denom

            # Calculate weights: sum alpha * ReLU(gradients) over spatial dimensions
            # Ensure gradients are positive using ReLU
            weights = (alpha * torch.relu(self.gradients)).sum(dim=(2, 3), keepdim=True)

            # Calculate CAM: sum weights * activations over the channel dimension
            cam_raw = (weights * self.activation).sum(dim=1, keepdim=True)
            cam_raw = torch.relu(cam_raw) # Apply ReLU to the final CAM

            if cam_raw.numel() > 0:
                 cam = cam_raw.squeeze().cpu().numpy()
            else:
                 print("Warning: Raw CAM tensor is empty.")
                 h, w = self.activation.shape[2:]
                 cam = np.zeros((h, w), dtype=np.float32)

        elif heatmap_method == 'augmented_gradcam++':
            gradients_pow2 = self.gradients.pow(2)
             # Denominator: 2 * grad^2 + sum(grad) - This looks unusual. Sum is usually over spatial dims.
             # Let's assume sum over spatial dims as in GradCAM++ for consistency.
            sum_grad_spatial = self.gradients.sum(dim=(2, 3), keepdim=True)
            alpha_denom = 2.0 * gradients_pow2 + sum_grad_spatial + 1e-8 # Added epsilon
            alpha_num = gradients_pow2
            alpha = alpha_num / alpha_denom

            weights = (alpha * torch.relu(self.gradients)).sum(dim=(2, 3), keepdim=True)

            cam_raw = (weights * self.activation).sum(dim=1, keepdim=True)
            # Missing ReLU on final CAM in original code snippet
            cam_raw = torch.relu(cam_raw)

            if cam_raw.numel() > 0:
                cam = cam_raw.squeeze().cpu().numpy()
            else:
                print("Warning: Raw CAM tensor is empty.")
                h, w = self.activation.shape[2:]
                cam = np.zeros((h, w), dtype=np.float32)

        else:
            raise ValueError(f"Unsupported method: {heatmap_method}")

        # Normalize
        eps = 1e-8

        if cam.max() > cam.min():
            cam_normalized = (cam - cam.min()) / (cam.max() - cam.min() + eps)
        elif cam.max() > 0: # Handle case where cam is constant but non-zero
            cam_normalized = cam / cam.max() # Normalize to 0 or 1
        else:
            cam_normalized = cam

        self.remove_hooks()

        self.activation = None
        self.gradients = None
        if input_tensor.grad is not None:
            input_tensor.grad.zero_() # Zero gradients on the input tensor


        return cam_normalized

    def remove_hooks(self):
         """Removes the forward and backward hooks."""
         if hasattr(self, 'forward_handle') and self.forward_handle:
             self.forward_handle.remove()
             self.forward_handle = None
         if hasattr(self, 'backward_handle') and self.backward_handle:
             self.backward_handle.remove()
             self.backward_handle = None


    def apply_heatmap(self, original_image, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
        if heatmap is None or heatmap.size == 0:
             print("Warning: Cannot apply empty heatmap.")
             return original_image

        # Ensure original image is 8-bit
        if original_image.dtype != np.uint8:
            # Assuming original image pixels are in [0, 1] float range if not uint8
            if original_image.max() <= 1.0:
                original_image = (original_image * 255).astype(np.uint8)
            else: # Otherwise, just try converting type, might need clipping
                 original_image = original_image.astype(np.uint8)


        h, w, _ = original_image.shape
        # Ensure heatmap is float32 for resize, handle potential 1D heatmap
        if heatmap.ndim == 1:
             # Attempt to reshape if it's a flattened square, otherwise error
             side = int(np.sqrt(heatmap.shape[0]))
             if side * side == heatmap.shape[0]:
                 heatmap = heatmap.reshape((side, side))
             else:
                 print(f"Warning: Cannot reshape 1D heatmap of size {heatmap.shape[0]} into 2D.")
                 return original_image # Cannot proceed

        # Resize heatmap
        heatmap_resized = cv2.resize(heatmap.astype(np.float32), (w, h))

        # Apply colormap - Ensure input to applyColorMap is uint8 [0, 255]
        heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), colormap)

        # Blend original image with heatmap
        overlaid_image = cv2.addWeighted(original_image, 1 - alpha, heatmap_colored, alpha, 0)

        return overlaid_image

    def apply_bounding_box(self, original_image, heatmap, threshold=0.5, min_contour_area=50, use_morph_close=True, kernel_size=5):
        if heatmap is None or heatmap.size == 0:
            print("Warning: Cannot apply bounding box to empty heatmap.")
            return original_image

        # Ensure original image is 8-bit BGR
        if original_image.dtype != np.uint8:
             if original_image.max() <= 1.0: # Assume float [0, 1]
                 image_uint8 = (original_image * 255).astype(np.uint8)
             else: # Assume float [0, 255] or other scale, just convert
                 image_uint8 = original_image.astype(np.uint8)
        else:
             image_uint8 = original_image.copy() # Work on a copy

        # Ensure image is 3 channels for drawing colored rectangle
        if image_uint8.ndim == 2:
             image_uint8 = cv2.cvtColor(image_uint8, cv2.COLOR_GRAY2BGR)
        elif image_uint8.shape[2] == 4: # Handle RGBA
             image_uint8 = cv2.cvtColor(image_uint8, cv2.COLOR_BGRA2BGR)


        h, w, _ = image_uint8.shape

        # Resize heatmap and ensure it's float32
        if heatmap.ndim == 1: # Handle flattened heatmap case
             side = int(np.sqrt(heatmap.shape[0]))
             if side * side == heatmap.shape[0]:
                 heatmap = heatmap.reshape((side, side))
             else:
                 print(f"Warning: Cannot reshape 1D heatmap of size {heatmap.shape[0]} for bounding box.")
                 return image_uint8

        heatmap_resized = cv2.resize(heatmap.astype(np.float32), (w, h))

        # Apply threshold to get binary map [0, 255]
        # Ensure the input heatmap is scaled 0-1 before multiplying by 255
        if heatmap_resized.max() > 1.0: # Check if already potentially [0, 255]
             heatmap_norm = heatmap_resized / 255.0
        else:
             heatmap_norm = heatmap_resized

        _, binary_map = cv2.threshold(np.uint8(255 * heatmap_norm), int(threshold * 255), 255, cv2.THRESH_BINARY)

        # Optional: Morphological closing to connect nearby regions
        if use_morph_close:
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            binary_map = cv2.morphologyEx(binary_map, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv2.findContours(binary_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Variables to store the overall bounding box coordinates
        overall_min_x, overall_min_y = w, h # Initialize with image dimensions
        overall_max_x, overall_max_y = 0, 0
        found_significant_contour = False

        # Iterate through contours to find the overall bounding box
        for contour in contours:
            # Filter based on contour area
            area = cv2.contourArea(contour)
            if area >= min_contour_area:
                found_significant_contour = True
                # Get bounding box for this contour
                x, y, wb, hb = cv2.boundingRect(contour)
                # Update overall coordinates
                overall_min_x = min(overall_min_x, x)
                overall_min_y = min(overall_min_y, y)
                overall_max_x = max(overall_max_x, x + wb)
                overall_max_y = max(overall_max_y, y + hb)

        # Draw the single overall bounding box if any significant contours were found
        if found_significant_contour:
            cv2.rectangle(image_uint8, (overall_min_x, overall_min_y), (overall_max_x, overall_max_y), (0, 255, 0), 2) # Green box, thickness 2
            # Optional: Add label or confidence if available
            # label = f"Detection: Conf {confidence:.2f}"
            # cv2.putText(image_uint8, label, (overall_min_x, overall_min_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        return image_uint8

    # --- Alternative using connectedComponentsWithStats ---
    def apply_bounding_box_connected_components(self, original_image, heatmap, threshold=0.5, min_component_area=50, use_morph_close=True, kernel_size=5):
        """
        Alternative method using connectedComponentsWithStats to draw a single bounding box.
        """
        if heatmap is None or heatmap.size == 0:
             print("Warning: Cannot apply bounding box to empty heatmap.")
             return original_image

        # Basic image prep (similar to the other method)
        if original_image.dtype != np.uint8:
             if original_image.max() <= 1.0: image_uint8 = (original_image * 255).astype(np.uint8)
             else: image_uint8 = original_image.astype(np.uint8)
        else: image_uint8 = original_image.copy()

        if image_uint8.ndim == 2: image_uint8 = cv2.cvtColor(image_uint8, cv2.COLOR_GRAY2BGR)
        elif image_uint8.shape[2] == 4: image_uint8 = cv2.cvtColor(image_uint8, cv2.COLOR_BGRA2BGR)

        h, w, _ = image_uint8.shape

        # Heatmap prep (similar to the other method)
        if heatmap.ndim == 1:
             side = int(np.sqrt(heatmap.shape[0]))
             if side * side == heatmap.shape[0]: heatmap = heatmap.reshape((side, side))
             else: return image_uint8
        heatmap_resized = cv2.resize(heatmap.astype(np.float32), (w, h))
        if heatmap_resized.max() > 1.0: heatmap_norm = heatmap_resized / 255.0
        else: heatmap_norm = heatmap_resized
        _, binary_map = cv2.threshold(np.uint8(255 * heatmap_norm), int(threshold * 255), 255, cv2.THRESH_BINARY)

        if use_morph_close:
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            binary_map = cv2.morphologyEx(binary_map, cv2.MORPH_CLOSE, kernel)

        # Find connected components
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_map, connectivity=8)

        # Variables for overall bounding box
        overall_min_x, overall_min_y = w, h
        overall_max_x, overall_max_y = 0, 0
        found_significant_component = False

        # Iterate through components (label 0 is the background)
        for i in range(1, num_labels):
            area = stats[i, cv2.CC_STAT_AREA]
            if area >= min_component_area:
                found_significant_component = True
                x = stats[i, cv2.CC_STAT_LEFT]
                y = stats[i, cv2.CC_STAT_TOP]
                wb = stats[i, cv2.CC_STAT_WIDTH]
                hb = stats[i, cv2.CC_STAT_HEIGHT]

                # Update overall coordinates
                overall_min_x = min(overall_min_x, x)
                overall_min_y = min(overall_min_y, y)
                overall_max_x = max(overall_max_x, x + wb)
                overall_max_y = max(overall_max_y, y + hb)

        # Draw the single overall bounding box
        if found_significant_component:
            cv2.rectangle(image_uint8, (overall_min_x, overall_min_y), (overall_max_x, overall_max_y), (0, 255, 0), 2)

        return image_uint8
    

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
def apply_gradcam(model_path, image_path, threshold, heatmap_method, alpha):
    try:
        # Load model
        model = load_model(model_path)
        
        # Find target layer dynamically
        target_layer = find_last_convolutional_layer(model)
        
        # Read and preprocess image
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Could not read image: {image_path}")
            return None

        # Convert to RGB if needed
        if len(original_image.shape) == 2 or original_image.shape[2] == 1:
            original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2RGB)
        elif original_image.shape[2] == 4:
            original_image = cv2.cvtColor(original_image, cv2.COLOR_RGBA2RGB)

        # Prepare input tensor
        input_tensor = transform(Image.fromarray(original_image)).unsqueeze(0)
        
        # Apply Grad-CAM
        gradcam = GradCAM(model, target_layer)
        heatmap = gradcam.generate_heatmap(input_tensor, heatmap_method)
        if heatmap is None:
             print(f"Failed to generate heatmap for {image_path}")
             return None
        overlay_image = gradcam.apply_heatmap(
            original_image, 
            heatmap, 
            alpha=alpha
        )
        # Create overlay and bounding boxes
        result_image = gradcam.apply_bounding_box(overlay_image, heatmap, threshold)
        return result_image
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        traceback.print_exc()
        return None

In [8]:
def process_dataset(model_paths, root_dir, output_base_dir, threshold=0.5, heatmap_method='default', alpha=0.4):
    print(f"--- Starting GradCam Process ---")

    for model_path in model_paths:
        # Create output directory for this model
        model_name = os.path.splitext(os.path.basename(model_path))[0]
        output_dir = os.path.join(output_base_dir, model_name)
        os.makedirs(output_dir, exist_ok=True)
        print(f"\nProcessing model: {model_name}")
        print(f"Output directory: {output_dir}")

        image_count = 0 # Reset image count for each model

        # Iterate through class directories
        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_path):
                continue

            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            # Iterate through images in the class directory
            for img_name in os.listdir(class_path):
                # Check if the maximum image count for this model has been reached

                # Skip files that don't start with 'orig_'
                if not img_name.startswith("orig_"):
                    continue

                img_path = os.path.join(class_path, img_name)
                # Check if it's a valid image file
                if not img_path.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue

                # --- Direct function call instead of submitting to executor ---
                try:
                    # print(f"Applying Grad-CAM to: {img_path}") # Optional: more verbose logging
                    result_image = apply_gradcam(model_path, img_path, threshold, heatmap_method, alpha)

                    # --- Process result immediately ---
                    if result_image is not None:
                        output_path = os.path.join(output_class_dir, img_name)
                        cv2.imwrite(output_path, result_image)
                        print(f"Processed and saved: {output_path} ({image_count + 1})")
                    else:
                         print(f"Skipped saving (result was None): {img_path}")

                    image_count += 1 # Increment counter only after processing an eligible image

                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    # Decide if you want to continue or stop on error
                    # continue
    print("\n--- Sequential Processing Finished ---")

In [9]:
model_paths = [
    # "../model/efficientnet_coffee.pth",
    # "../model/convnext_tiny_coffee.pth",
    # "../model/mobilenetv2_coffee.pth",
    # "../model/regnety_8gf_coffee.pth",
    "../model/resnet50_coffee_no_au.pth",
    # "../model/resnet50_coffee.pth"
]

In [10]:
threshold= 0.5

In [11]:
for n, model_path in enumerate(model_paths):
    output_path = f"../data/bounding_box_gradcam_{threshold}_augmented_gradcam++/val/"
    
    process_dataset(
        [model_path], 
        "../data/Final_CLD_data/val", 
        output_path, 
        threshold=threshold,
        heatmap_method='augmented_gradcam++',  # 'default', 'guided', 'gradcam++', 'augmented_gradcam++'
        alpha=0.3
    )

--- Starting GradCam Process ---

Processing model: resnet50_coffee_no_au
Output directory: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au


C:\Users\quanb\AppData\Local\Temp\ipykernel_26496\2729930924.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (10).jpg (1)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1000).jpg (2)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1020).jpg (3)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1025).jpg (4)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1029).jpg (5)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1089).jpg (6)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_gradcam++/val/resnet50_coffee_no_au\Cerscospora\orig_4 (1094).jpg (7)
Processed and saved: ../data/bounding_box_gradcam_0.5_augmented_